# Fixed Income Securities

A comprehensive guide to fixed income securities, bond pricing, duration, convexity, and interest rate risk management.

## Table of Contents
1. [Overview of Fixed Income Securities](#1.-Overview-of-Fixed-Income-Securities)
2. [Bond Pricing and Yield Calculations](#2.-Bond-Pricing-and-Yield-Calculations)
3. [Duration](#3.-Duration)
4. [Convexity](#4.-Convexity)
5. [Term Structure of Interest Rates](#5.-Term-Structure-of-Interest-Rates)
6. [Bond Portfolio Management](#6.-Bond-Portfolio-Management)
7. [Credit Risk in Fixed Income](#7.-Credit-Risk-in-Fixed-Income)
8. [Interest Rate Risk Hedging](#8.-Interest-Rate-Risk-Hedging)
9. [Comprehensive Examples](#9.-Comprehensive-Examples)

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from scipy.optimize import brentq
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

---
## 1. Overview of Fixed Income Securities

Fixed income securities are debt instruments that provide investors with a stream of fixed periodic payments and the return of principal at maturity.

### Types of Fixed Income Securities

| Security Type | Maturity | Coupon | Issued By |
|--------------|----------|--------|----------|
| **Treasury Bills (T-Bills)** | < 1 year | Zero-coupon (discount) | Government |
| **Treasury Notes (T-Notes)** | 2-10 years | Semi-annual | Government |
| **Treasury Bonds (T-Bonds)** | 10-30 years | Semi-annual | Government |
| **Corporate Bonds** | Various | Semi-annual/Annual | Corporations |
| **Municipal Bonds** | Various | Semi-annual | State/Local Gov |
| **Agency Bonds** | Various | Various | GSEs (Fannie Mae, etc.) |

### Key Bond Terminology

- **Face Value (Par Value)**: The principal amount paid at maturity (typically $1,000)
- **Coupon Rate**: Annual interest rate as a percentage of face value
- **Maturity Date**: Date when principal is repaid
- **Current Yield**: Annual coupon / Current market price
- **Yield to Maturity (YTM)**: Internal rate of return if held to maturity

### Bond Price Quotation

Bonds are typically quoted as a percentage of par value:
- **At Par**: Price = 100 (trading at face value)
- **At Premium**: Price > 100 (coupon > market yield)
- **At Discount**: Price < 100 (coupon < market yield)

---
## 2. Bond Pricing and Yield Calculations

### Bond Pricing Formula

The price of a bond is the present value of all future cash flows:

$$P = \sum_{t=1}^{n} \frac{C}{(1+y)^t} + \frac{F}{(1+y)^n}$$

Where:
- $P$ = Bond price
- $C$ = Coupon payment per period
- $F$ = Face value
- $y$ = Yield per period
- $n$ = Number of periods to maturity

For a semi-annual bond:

$$P = \sum_{t=1}^{2T} \frac{C/2}{(1+y/2)^t} + \frac{F}{(1+y/2)^{2T}}$$

### Types of Yields

#### Current Yield
$$\text{Current Yield} = \frac{\text{Annual Coupon Payment}}{\text{Current Market Price}}$$

#### Yield to Maturity (YTM)
The discount rate that equates the present value of cash flows to the current price.

#### Yield to Call (YTC)
For callable bonds, the yield assuming the bond is called at the first call date:

$$P = \sum_{t=1}^{n_c} \frac{C}{(1+y_c)^t} + \frac{\text{Call Price}}{(1+y_c)^{n_c}}$$

#### Yield to Worst (YTW)
The minimum of YTM, YTC, and yields to all other call dates.

In [ ]:
class Bond:
    """
    A class to represent a fixed-rate bond and perform various calculations.
    
    Parameters:
    -----------
    face_value : float
        The par/face value of the bond
    coupon_rate : float
        Annual coupon rate as a decimal (e.g., 0.05 for 5%)
    years_to_maturity : float
        Time to maturity in years
    frequency : int
        Number of coupon payments per year (1=annual, 2=semi-annual)
    """
    
    def __init__(self, face_value=1000, coupon_rate=0.05, years_to_maturity=10, frequency=2):
        self.face_value = face_value
        self.coupon_rate = coupon_rate
        self.years_to_maturity = years_to_maturity
        self.frequency = frequency
        self.periods = int(years_to_maturity * frequency)
        self.coupon_payment = (coupon_rate * face_value) / frequency
    
    def price(self, ytm):
        """
        Calculate bond price given yield to maturity.
        
        Parameters:
        -----------
        ytm : float
            Annual yield to maturity as a decimal
        
        Returns:
        --------
        float : Bond price
        """
        y = ytm / self.frequency
        n = self.periods
        
        if y == 0:
            return self.coupon_payment * n + self.face_value
        
        # Present value of coupons (annuity formula)
        pv_coupons = self.coupon_payment * (1 - (1 + y)**(-n)) / y
        
        # Present value of face value
        pv_face = self.face_value / (1 + y)**n
        
        return pv_coupons + pv_face
    
    def ytm(self, price):
        """
        Calculate yield to maturity given price using numerical methods.
        
        Parameters:
        -----------
        price : float
            Current market price of the bond
        
        Returns:
        --------
        float : Annual yield to maturity
        """
        def objective(y):
            return self.price(y) - price
        
        # Use Brent's method to find the root
        return brentq(objective, -0.5, 1.0)
    
    def current_yield(self, price):
        """
        Calculate current yield.
        
        Parameters:
        -----------
        price : float
            Current market price of the bond
        
        Returns:
        --------
        float : Current yield as a decimal
        """
        annual_coupon = self.coupon_rate * self.face_value
        return annual_coupon / price
    
    def yield_to_call(self, price, call_price, years_to_call):
        """
        Calculate yield to call for callable bonds.
        
        Parameters:
        -----------
        price : float
            Current market price
        call_price : float
            Price at which bond can be called
        years_to_call : float
            Years until first call date
        
        Returns:
        --------
        float : Annual yield to call
        """
        periods_to_call = int(years_to_call * self.frequency)
        
        def objective(y):
            y_per = y / self.frequency
            pv_coupons = self.coupon_payment * (1 - (1 + y_per)**(-periods_to_call)) / y_per
            pv_call = call_price / (1 + y_per)**periods_to_call
            return pv_coupons + pv_call - price
        
        return brentq(objective, -0.5, 1.0)

In [ ]:
# Example: Bond Pricing and Yield Calculations
print("=" * 60)
print("BOND PRICING AND YIELD CALCULATIONS")
print("=" * 60)

# Create a bond
bond = Bond(
    face_value=1000,
    coupon_rate=0.06,  # 6% annual coupon
    years_to_maturity=10,
    frequency=2  # Semi-annual payments
)

# Calculate prices at different yields
yields = [0.04, 0.05, 0.06, 0.07, 0.08]
print("\n📊 Bond Prices at Different Yields:")
print("-" * 40)
print(f"{'YTM':>8} {'Price':>12} {'Status':>12}")
print("-" * 40)

for y in yields:
    p = bond.price(y)
    status = "At Par" if abs(p - 1000) < 0.01 else ("Premium" if p > 1000 else "Discount")
    print(f"{y*100:>7.2f}% ${p:>10.2f} {status:>12}")

# Calculate YTM from price
market_price = 950
calculated_ytm = bond.ytm(market_price)
current_yld = bond.current_yield(market_price)

print(f"\n📈 Yield Calculations for Price = ${market_price}:")
print("-" * 40)
print(f"Yield to Maturity (YTM): {calculated_ytm*100:.4f}%")
print(f"Current Yield:           {current_yld*100:.4f}%")

# Yield to Call example
ytc = bond.yield_to_call(price=1050, call_price=1030, years_to_call=5)
print(f"\n📞 Callable Bond (Price=$1050, Call=$1030, 5 years):")
print(f"Yield to Call: {ytc*100:.4f}%")

In [ ]:
# Visualize Price-Yield Relationship
yields_range = np.linspace(0.01, 0.12, 100)
prices = [bond.price(y) for y in yields_range]

# Create bonds with different maturities for comparison
bond_5yr = Bond(face_value=1000, coupon_rate=0.06, years_to_maturity=5, frequency=2)
bond_10yr = Bond(face_value=1000, coupon_rate=0.06, years_to_maturity=10, frequency=2)
bond_30yr = Bond(face_value=1000, coupon_rate=0.06, years_to_maturity=30, frequency=2)

prices_5yr = [bond_5yr.price(y) for y in yields_range]
prices_10yr = [bond_10yr.price(y) for y in yields_range]
prices_30yr = [bond_30yr.price(y) for y in yields_range]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=yields_range * 100, y=prices_5yr,
    mode='lines', name='5-Year Bond',
    line=dict(color='#2ecc71', width=2)
))

fig.add_trace(go.Scatter(
    x=yields_range * 100, y=prices_10yr,
    mode='lines', name='10-Year Bond',
    line=dict(color='#3498db', width=2)
))

fig.add_trace(go.Scatter(
    x=yields_range * 100, y=prices_30yr,
    mode='lines', name='30-Year Bond',
    line=dict(color='#e74c3c', width=2)
))

# Add par value line
fig.add_hline(y=1000, line_dash="dash", line_color="gray", 
              annotation_text="Par Value")

fig.update_layout(
    title='Price-Yield Relationship (6% Coupon Bonds)',
    xaxis_title='Yield to Maturity (%)',
    yaxis_title='Bond Price ($)',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99)
)

fig

---
## 3. Duration

Duration measures a bond's sensitivity to interest rate changes. It represents the weighted average time to receive cash flows.

### Macaulay Duration

The weighted average time to receive cash flows:

$$D_{Mac} = \frac{\sum_{t=1}^{n} t \cdot \frac{CF_t}{(1+y)^t}}{P}$$

Where:
- $CF_t$ = Cash flow at time $t$
- $y$ = Yield per period
- $P$ = Bond price

### Modified Duration

Measures the percentage price change for a 1% change in yield:

$$D_{Mod} = \frac{D_{Mac}}{1 + y/m}$$

Where $m$ = number of compounding periods per year.

**Price Change Approximation:**
$$\frac{\Delta P}{P} \approx -D_{Mod} \cdot \Delta y$$

### Effective Duration

Used for bonds with embedded options (callable, putable):

$$D_{Eff} = \frac{P_{-\Delta y} - P_{+\Delta y}}{2 \cdot P_0 \cdot \Delta y}$$

Where:
- $P_{-\Delta y}$ = Price if yield decreases by $\Delta y$
- $P_{+\Delta y}$ = Price if yield increases by $\Delta y$
- $P_0$ = Current price

### Dollar Duration (DV01)

The dollar change in price for a 1 basis point (0.01%) change in yield:

$$DV01 = D_{Mod} \cdot P \cdot 0.0001$$

In [ ]:
def calculate_duration(bond, ytm):
    """
    Calculate Macaulay Duration, Modified Duration, and DV01.
    
    Parameters:
    -----------
    bond : Bond
        Bond object
    ytm : float
        Yield to maturity as a decimal
    
    Returns:
    --------
    dict : Dictionary containing duration measures
    """
    y = ytm / bond.frequency
    price = bond.price(ytm)
    
    # Calculate weighted present value of cash flows
    weighted_pv = 0
    for t in range(1, bond.periods + 1):
        if t < bond.periods:
            cf = bond.coupon_payment
        else:
            cf = bond.coupon_payment + bond.face_value
        
        pv = cf / (1 + y)**t
        weighted_pv += t * pv
    
    # Macaulay Duration (in periods, then convert to years)
    macaulay_duration = weighted_pv / price / bond.frequency
    
    # Modified Duration
    modified_duration = macaulay_duration / (1 + ytm / bond.frequency)
    
    # DV01 (Dollar Value of 01 = 1 basis point)
    dv01 = modified_duration * price * 0.0001
    
    return {
        'macaulay_duration': macaulay_duration,
        'modified_duration': modified_duration,
        'dv01': dv01,
        'price': price
    }


def calculate_effective_duration(bond, ytm, delta_y=0.0001):
    """
    Calculate Effective Duration using numerical approximation.
    
    Parameters:
    -----------
    bond : Bond
        Bond object
    ytm : float
        Yield to maturity as a decimal
    delta_y : float
        Yield change for numerical differentiation
    
    Returns:
    --------
    float : Effective duration
    """
    p0 = bond.price(ytm)
    p_up = bond.price(ytm + delta_y)
    p_down = bond.price(ytm - delta_y)
    
    effective_duration = (p_down - p_up) / (2 * p0 * delta_y)
    return effective_duration

In [ ]:
# Example: Duration Calculations
print("=" * 60)
print("DURATION CALCULATIONS")
print("=" * 60)

# Create bonds with different characteristics
bonds = [
    Bond(1000, 0.04, 5, 2),   # 4% coupon, 5-year
    Bond(1000, 0.06, 10, 2),  # 6% coupon, 10-year
    Bond(1000, 0.08, 20, 2),  # 8% coupon, 20-year
    Bond(1000, 0.00, 10, 2),  # Zero coupon, 10-year
]

ytm = 0.05  # 5% yield

print(f"\n📊 Duration Comparison at YTM = {ytm*100}%")
print("-" * 75)
print(f"{'Bond':>20} {'Price':>10} {'Mac Dur':>10} {'Mod Dur':>10} {'DV01':>10}")
print("-" * 75)

duration_data = []
for b in bonds:
    dur = calculate_duration(b, ytm)
    eff_dur = calculate_effective_duration(b, ytm)
    coupon_type = "Zero" if b.coupon_rate == 0 else f"{b.coupon_rate*100:.0f}%"
    bond_name = f"{coupon_type} {b.years_to_maturity}yr"
    print(f"{bond_name:>20} ${dur['price']:>8.2f} {dur['macaulay_duration']:>10.4f} {dur['modified_duration']:>10.4f} ${dur['dv01']:>8.4f}")
    
    duration_data.append({
        'Bond': bond_name,
        'Maturity': b.years_to_maturity,
        'Coupon': b.coupon_rate,
        'Macaulay Duration': dur['macaulay_duration'],
        'Modified Duration': dur['modified_duration'],
        'Effective Duration': eff_dur
    })

# Show that zero-coupon bond duration equals maturity
print("\n💡 Key Insight: For a zero-coupon bond, Macaulay Duration = Maturity")

In [ ]:
# Visualize Duration vs Maturity and Coupon
maturities = np.arange(1, 31, 1)
coupons = [0.02, 0.04, 0.06, 0.08]
ytm_viz = 0.05

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Duration vs Maturity', 'Duration vs Coupon Rate')
)

# Plot 1: Duration vs Maturity for different coupons
colors = ['#e74c3c', '#3498db', '#2ecc71', '#9b59b6']
for coupon, color in zip(coupons, colors):
    durations = []
    for mat in maturities:
        b = Bond(1000, coupon, mat, 2)
        dur = calculate_duration(b, ytm_viz)
        durations.append(dur['modified_duration'])
    
    fig.add_trace(
        go.Scatter(x=maturities, y=durations, mode='lines',
                   name=f'{coupon*100:.0f}% Coupon', line=dict(color=color)),
        row=1, col=1
    )

# Add zero-coupon line (duration = maturity)
fig.add_trace(
    go.Scatter(x=maturities, y=maturities, mode='lines',
               name='Zero Coupon', line=dict(color='black', dash='dash')),
    row=1, col=1
)

# Plot 2: Duration vs Coupon Rate for different maturities
coupon_range = np.linspace(0.01, 0.12, 50)
maturity_options = [5, 10, 20, 30]

for mat, color in zip(maturity_options, colors):
    durations = []
    for c in coupon_range:
        b = Bond(1000, c, mat, 2)
        dur = calculate_duration(b, ytm_viz)
        durations.append(dur['modified_duration'])
    
    fig.add_trace(
        go.Scatter(x=coupon_range*100, y=durations, mode='lines',
                   name=f'{mat}-Year', line=dict(color=color), showlegend=False),
        row=1, col=2
    )

fig.update_xaxes(title_text='Maturity (Years)', row=1, col=1)
fig.update_xaxes(title_text='Coupon Rate (%)', row=1, col=2)
fig.update_yaxes(title_text='Modified Duration (Years)', row=1, col=1)
fig.update_yaxes(title_text='Modified Duration (Years)', row=1, col=2)

fig.update_layout(
    title=f'Duration Analysis at YTM = {ytm_viz*100}%',
    template='plotly_white',
    height=450,
    showlegend=True
)

fig

---
## 4. Convexity

Convexity measures the curvature in the price-yield relationship. It improves the accuracy of duration-based price change estimates.

### Convexity Formula

$$C = \frac{1}{P(1+y)^2} \sum_{t=1}^{n} \frac{t(t+1) \cdot CF_t}{(1+y)^t}$$

For annual compounding. For semi-annual:

$$C = \frac{1}{P} \sum_{t=1}^{n} \frac{t(t+1) \cdot CF_t}{(1+y/2)^{t+2}} \cdot \frac{1}{m^2}$$

### Dollar Convexity

$$\text{Dollar Convexity} = C \cdot P$$

### Price Change with Convexity Adjustment

$$\frac{\Delta P}{P} \approx -D_{Mod} \cdot \Delta y + \frac{1}{2} \cdot C \cdot (\Delta y)^2$$

### Key Properties of Convexity

1. **Convexity is always positive** for option-free bonds
2. **Higher convexity is desirable** - provides better price appreciation when yields fall
3. **Convexity increases with maturity** and decreases with coupon rate
4. **Callable bonds have negative convexity** at low yields due to call risk

In [ ]:
def calculate_convexity(bond, ytm):
    """
    Calculate convexity of a bond.
    
    Parameters:
    -----------
    bond : Bond
        Bond object
    ytm : float
        Yield to maturity as a decimal
    
    Returns:
    --------
    float : Convexity (in years squared)
    """
    y = ytm / bond.frequency
    price = bond.price(ytm)
    
    convexity_sum = 0
    for t in range(1, bond.periods + 1):
        if t < bond.periods:
            cf = bond.coupon_payment
        else:
            cf = bond.coupon_payment + bond.face_value
        
        pv = cf / (1 + y)**(t + 2)
        convexity_sum += t * (t + 1) * pv
    
    # Adjust for compounding frequency
    convexity = convexity_sum / (price * bond.frequency**2)
    
    return convexity


def price_change_with_convexity(bond, ytm, delta_y):
    """
    Calculate price change using duration and convexity.
    
    Parameters:
    -----------
    bond : Bond
        Bond object
    ytm : float
        Current yield to maturity
    delta_y : float
        Change in yield
    
    Returns:
    --------
    dict : Dictionary with price change estimates
    """
    dur = calculate_duration(bond, ytm)
    convexity = calculate_convexity(bond, ytm)
    price = dur['price']
    mod_dur = dur['modified_duration']
    
    # Duration-only estimate
    pct_change_duration = -mod_dur * delta_y
    
    # Duration + Convexity estimate
    pct_change_with_convexity = -mod_dur * delta_y + 0.5 * convexity * delta_y**2
    
    # Actual price change
    new_price = bond.price(ytm + delta_y)
    actual_pct_change = (new_price - price) / price
    
    return {
        'original_price': price,
        'new_price': new_price,
        'actual_change': actual_pct_change * 100,
        'duration_estimate': pct_change_duration * 100,
        'convexity_estimate': pct_change_with_convexity * 100,
        'duration_error': (pct_change_duration - actual_pct_change) * 100,
        'convexity_error': (pct_change_with_convexity - actual_pct_change) * 100,
        'convexity': convexity
    }

In [ ]:
# Example: Convexity Analysis
print("=" * 70)
print("CONVEXITY ANALYSIS")
print("=" * 70)

# Test bond
test_bond = Bond(1000, 0.06, 20, 2)  # 6% coupon, 20-year
ytm_base = 0.06

dur = calculate_duration(test_bond, ytm_base)
convex = calculate_convexity(test_bond, ytm_base)

print(f"\n📊 Bond Characteristics (6% 20-Year at 6% YTM):")
print("-" * 50)
print(f"Price:              ${dur['price']:.2f}")
print(f"Modified Duration:  {dur['modified_duration']:.4f} years")
print(f"Convexity:          {convex:.4f} years²")

# Compare price changes for different yield movements
print(f"\n📈 Price Change Comparison (Duration vs Duration+Convexity):")
print("-" * 70)
print(f"{'Δ Yield':>10} {'Actual %':>12} {'Dur Only %':>12} {'Dur+Conv %':>12} {'Conv Adj':>12}")
print("-" * 70)

yield_changes = [-0.02, -0.01, -0.005, 0.005, 0.01, 0.02]
for dy in yield_changes:
    result = price_change_with_convexity(test_bond, ytm_base, dy)
    conv_adj = result['convexity_estimate'] - result['duration_estimate']
    print(f"{dy*100:>+9.1f}% {result['actual_change']:>+11.4f}% {result['duration_estimate']:>+11.4f}% {result['convexity_estimate']:>+11.4f}% {conv_adj:>+11.4f}%")

print("\n💡 Notice: Convexity adjustment is always positive, improving estimates for both yield increases and decreases.")

In [ ]:
# Visualize Duration and Convexity Effects
yield_range = np.linspace(-0.03, 0.03, 100)
base_ytm = 0.06

# Calculate all price changes
actual_prices = [test_bond.price(base_ytm + dy) for dy in yield_range]
actual_changes = [(p - 1000) / 1000 * 100 for p in actual_prices]

# Duration-only approximation
mod_dur = dur['modified_duration']
duration_changes = [-mod_dur * dy * 100 for dy in yield_range]

# Duration + Convexity approximation
convexity_changes = [(-mod_dur * dy + 0.5 * convex * dy**2) * 100 for dy in yield_range]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=yield_range * 100, y=actual_changes,
    mode='lines', name='Actual Price Change',
    line=dict(color='#2c3e50', width=3)
))

fig.add_trace(go.Scatter(
    x=yield_range * 100, y=duration_changes,
    mode='lines', name='Duration Approximation',
    line=dict(color='#e74c3c', width=2, dash='dash')
))

fig.add_trace(go.Scatter(
    x=yield_range * 100, y=convexity_changes,
    mode='lines', name='Duration + Convexity',
    line=dict(color='#27ae60', width=2, dash='dot')
))

fig.add_vline(x=0, line_dash="solid", line_color="gray", line_width=1)
fig.add_hline(y=0, line_dash="solid", line_color="gray", line_width=1)

fig.update_layout(
    title='Price Change Approximation: Duration vs Duration + Convexity',
    xaxis_title='Change in Yield (%)',
    yaxis_title='Change in Price (%)',
    template='plotly_white',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    height=500
)

fig

---
## 5. Term Structure of Interest Rates

The term structure describes the relationship between interest rates and time to maturity.

### Spot Rates (Zero Rates)

The yield on a zero-coupon bond maturing at time $t$:

$$P_t = \frac{100}{(1 + z_t)^t}$$

Where $z_t$ is the spot rate for maturity $t$.

### Forward Rates

The implied future interest rate between two periods:

$$f_{t_1, t_2} = \left[ \frac{(1 + z_{t_2})^{t_2}}{(1 + z_{t_1})^{t_1}} \right]^{\frac{1}{t_2 - t_1}} - 1$$

### Yield Curve Shapes

1. **Normal (Upward Sloping)**: Long-term rates > Short-term rates
   - Indicates economic expansion expectations

2. **Inverted (Downward Sloping)**: Short-term rates > Long-term rates
   - Often precedes recessions

3. **Flat**: Similar rates across maturities
   - Economic uncertainty

4. **Humped**: Mid-term rates highest
   - Transition period

### Theories of Term Structure

| Theory | Description |
|--------|-------------|
| **Expectations Theory** | Forward rates = Expected future spot rates |
| **Liquidity Preference** | Long-term rates include liquidity premium |
| **Market Segmentation** | Rates determined by supply/demand in each segment |
| **Preferred Habitat** | Investors have maturity preferences but will shift for adequate premium |

In [ ]:
class YieldCurve:
    """
    A class to model and analyze yield curves.
    """
    
    def __init__(self, maturities, spot_rates):
        """
        Initialize yield curve with maturities and spot rates.
        
        Parameters:
        -----------
        maturities : array-like
            Maturities in years
        spot_rates : array-like
            Corresponding spot rates as decimals
        """
        self.maturities = np.array(maturities)
        self.spot_rates = np.array(spot_rates)
    
    def interpolate_rate(self, t):
        """Linearly interpolate spot rate for any maturity."""
        return np.interp(t, self.maturities, self.spot_rates)
    
    def forward_rate(self, t1, t2):
        """
        Calculate forward rate between t1 and t2.
        
        Parameters:
        -----------
        t1 : float
            Start time in years
        t2 : float
            End time in years
        
        Returns:
        --------
        float : Forward rate as a decimal
        """
        z1 = self.interpolate_rate(t1)
        z2 = self.interpolate_rate(t2)
        
        return ((1 + z2)**t2 / (1 + z1)**t1)**(1/(t2-t1)) - 1
    
    def discount_factor(self, t):
        """Calculate discount factor for maturity t."""
        z = self.interpolate_rate(t)
        return 1 / (1 + z)**t
    
    def par_yield(self, maturity, frequency=2):
        """
        Calculate par yield for a given maturity.
        Par yield is the coupon rate that makes a bond trade at par.
        """
        periods = int(maturity * frequency)
        
        # Sum of discount factors
        sum_df = sum(self.discount_factor(t/frequency) for t in range(1, periods + 1))
        
        # Par yield formula
        final_df = self.discount_factor(maturity)
        par_yield = frequency * (1 - final_df) / sum_df
        
        return par_yield
    
    def get_forward_curve(self, start_times):
        """Get forward rates for 1-year periods starting at given times."""
        forward_rates = []
        for t in start_times:
            if t + 1 <= max(self.maturities):
                forward_rates.append(self.forward_rate(t, t + 1))
            else:
                forward_rates.append(np.nan)
        return np.array(forward_rates)

In [ ]:
# Example: Term Structure Analysis
print("=" * 60)
print("TERM STRUCTURE ANALYSIS")
print("=" * 60)

# Create sample yield curves
maturities = np.array([0.25, 0.5, 1, 2, 3, 5, 7, 10, 20, 30])

# Normal curve
normal_rates = np.array([0.025, 0.027, 0.030, 0.035, 0.040, 0.045, 0.048, 0.050, 0.052, 0.053])

# Inverted curve
inverted_rates = np.array([0.050, 0.048, 0.045, 0.040, 0.038, 0.035, 0.033, 0.032, 0.031, 0.030])

# Flat curve
flat_rates = np.array([0.040, 0.040, 0.040, 0.040, 0.040, 0.040, 0.040, 0.040, 0.040, 0.040])

# Create yield curve objects
normal_curve = YieldCurve(maturities, normal_rates)
inverted_curve = YieldCurve(maturities, inverted_rates)
flat_curve = YieldCurve(maturities, flat_rates)

# Calculate forward rates for normal curve
print("\n📊 Forward Rates from Normal Curve:")
print("-" * 50)
print(f"{'Period':>15} {'Forward Rate':>15}")
print("-" * 50)
for t in [0, 1, 2, 3, 5, 10]:
    if t + 1 <= 30:
        fwd = normal_curve.forward_rate(t, t + 1)
        print(f"{t}y to {t+1}y:  {fwd*100:>12.4f}%")

# Calculate par yields
print("\n📈 Par Yields from Normal Curve:")
print("-" * 50)
for mat in [2, 5, 10, 20, 30]:
    par = normal_curve.par_yield(mat)
    spot = normal_curve.interpolate_rate(mat)
    print(f"{mat:>2}-Year: Par Yield = {par*100:.4f}%, Spot Rate = {spot*100:.4f}%")

In [ ]:
# Visualize Yield Curves
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Yield Curve Shapes', 'Spot vs Forward Rates')
)

# Plot 1: Different yield curve shapes
mat_smooth = np.linspace(0.25, 30, 100)

fig.add_trace(
    go.Scatter(
        x=mat_smooth, 
        y=[normal_curve.interpolate_rate(t) * 100 for t in mat_smooth],
        mode='lines', name='Normal (Upward)',
        line=dict(color='#27ae60', width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=mat_smooth, 
        y=[inverted_curve.interpolate_rate(t) * 100 for t in mat_smooth],
        mode='lines', name='Inverted',
        line=dict(color='#e74c3c', width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=mat_smooth, 
        y=[flat_curve.interpolate_rate(t) * 100 for t in mat_smooth],
        mode='lines', name='Flat',
        line=dict(color='#3498db', width=2)
    ),
    row=1, col=1
)

# Plot 2: Spot vs Forward rates
forward_mats = np.arange(0, 29, 1)
spot_rates_plot = [normal_curve.interpolate_rate(t) * 100 for t in forward_mats]
forward_rates_plot = [normal_curve.forward_rate(t, t+1) * 100 for t in forward_mats]

fig.add_trace(
    go.Scatter(
        x=forward_mats, y=spot_rates_plot,
        mode='lines', name='Spot Rates',
        line=dict(color='#2c3e50', width=2),
        showlegend=True
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=forward_mats, y=forward_rates_plot,
        mode='lines', name='1Y Forward Rates',
        line=dict(color='#9b59b6', width=2, dash='dash'),
        showlegend=True
    ),
    row=1, col=2
)

fig.update_xaxes(title_text='Maturity (Years)', row=1, col=1)
fig.update_xaxes(title_text='Time (Years)', row=1, col=2)
fig.update_yaxes(title_text='Yield (%)', row=1, col=1)
fig.update_yaxes(title_text='Rate (%)', row=1, col=2)

fig.update_layout(
    title='Term Structure of Interest Rates',
    template='plotly_white',
    height=450,
    showlegend=True
)

fig

---
## 6. Bond Portfolio Management

### Portfolio Duration

The portfolio duration is the weighted average of individual bond durations:

$$D_P = \sum_{i=1}^{n} w_i \cdot D_i$$

Where:
- $w_i$ = Weight of bond $i$ (market value / total portfolio value)
- $D_i$ = Duration of bond $i$

### Immunization Strategies

#### Classical Immunization
Match portfolio duration to liability duration to protect against parallel yield curve shifts.

**Conditions:**
1. PV(Assets) ≥ PV(Liabilities)
2. Duration(Assets) = Duration(Liabilities)
3. Asset cash flows must bracket liability cash flows

#### Cash Flow Matching
Structure portfolio cash flows to exactly match liability cash flows.

### Active Strategies

| Strategy | Description | Risk |
|----------|-------------|------|
| **Duration Timing** | Adjust duration based on rate forecasts | Interest rate risk |
| **Yield Curve Trades** | Position for curve shape changes | Curve risk |
| **Sector Rotation** | Shift between government/corporate | Credit risk |
| **Credit Quality** | Move up/down the credit spectrum | Default risk |

### Yield Curve Strategies

- **Bullet**: Concentrate holdings around a single maturity
- **Barbell**: Concentrate at short and long ends
- **Ladder**: Equal allocation across maturities

In [ ]:
class BondPortfolio:
    """
    A class to manage and analyze a portfolio of bonds.
    """
    
    def __init__(self):
        self.bonds = []
        self.quantities = []
        self.ytms = []
    
    def add_bond(self, bond, quantity, ytm):
        """
        Add a bond to the portfolio.
        
        Parameters:
        -----------
        bond : Bond
            Bond object
        quantity : int
            Number of bonds
        ytm : float
            Yield to maturity of this bond
        """
        self.bonds.append(bond)
        self.quantities.append(quantity)
        self.ytms.append(ytm)
    
    def portfolio_value(self):
        """Calculate total portfolio market value."""
        return sum(
            bond.price(ytm) * qty 
            for bond, qty, ytm in zip(self.bonds, self.quantities, self.ytms)
        )
    
    def portfolio_duration(self):
        """Calculate portfolio modified duration."""
        total_value = self.portfolio_value()
        
        weighted_duration = 0
        for bond, qty, ytm in zip(self.bonds, self.quantities, self.ytms):
            dur = calculate_duration(bond, ytm)
            weight = (bond.price(ytm) * qty) / total_value
            weighted_duration += weight * dur['modified_duration']
        
        return weighted_duration
    
    def portfolio_convexity(self):
        """Calculate portfolio convexity."""
        total_value = self.portfolio_value()
        
        weighted_convexity = 0
        for bond, qty, ytm in zip(self.bonds, self.quantities, self.ytms):
            conv = calculate_convexity(bond, ytm)
            weight = (bond.price(ytm) * qty) / total_value
            weighted_convexity += weight * conv
        
        return weighted_convexity
    
    def portfolio_dv01(self):
        """Calculate portfolio DV01."""
        return self.portfolio_duration() * self.portfolio_value() * 0.0001
    
    def summary(self):
        """Generate portfolio summary."""
        data = []
        for i, (bond, qty, ytm) in enumerate(zip(self.bonds, self.quantities, self.ytms)):
            dur = calculate_duration(bond, ytm)
            conv = calculate_convexity(bond, ytm)
            data.append({
                'Bond': f"Bond {i+1}",
                'Coupon': f"{bond.coupon_rate*100:.1f}%",
                'Maturity': f"{bond.years_to_maturity}yr",
                'Quantity': qty,
                'Price': dur['price'],
                'Market Value': dur['price'] * qty,
                'Duration': dur['modified_duration'],
                'Convexity': conv
            })
        return pd.DataFrame(data)

In [ ]:
# Example: Portfolio Analysis - Compare Bullet, Barbell, and Ladder
print("=" * 70)
print("BOND PORTFOLIO STRATEGIES COMPARISON")
print("=" * 70)

# Common yield for all bonds
common_ytm = 0.05

# Bullet Portfolio: All in 10-year bonds
bullet_portfolio = BondPortfolio()
bullet_portfolio.add_bond(Bond(1000, 0.05, 10, 2), 100, common_ytm)

# Barbell Portfolio: 50% in 2-year, 50% in 30-year
barbell_portfolio = BondPortfolio()
barbell_portfolio.add_bond(Bond(1000, 0.05, 2, 2), 50, common_ytm)
barbell_portfolio.add_bond(Bond(1000, 0.05, 30, 2), 50, common_ytm)

# Ladder Portfolio: Equal amounts at 2, 5, 10, 20, 30 years
ladder_portfolio = BondPortfolio()
for mat in [2, 5, 10, 20, 30]:
    ladder_portfolio.add_bond(Bond(1000, 0.05, mat, 2), 20, common_ytm)

# Compare portfolios
portfolios = [
    ('Bullet (10yr)', bullet_portfolio),
    ('Barbell (2yr/30yr)', barbell_portfolio),
    ('Ladder (2-30yr)', ladder_portfolio)
]

print("\n📊 Portfolio Comparison:")
print("-" * 70)
print(f"{'Strategy':>20} {'Value':>15} {'Duration':>12} {'Convexity':>12} {'DV01':>12}")
print("-" * 70)

comparison_data = []
for name, port in portfolios:
    value = port.portfolio_value()
    dur = port.portfolio_duration()
    conv = port.portfolio_convexity()
    dv01 = port.portfolio_dv01()
    print(f"{name:>20} ${value:>13,.0f} {dur:>11.4f} {conv:>11.2f} ${dv01:>10,.2f}")
    comparison_data.append({'Strategy': name, 'Duration': dur, 'Convexity': conv})

print("\n💡 Key Observations:")
print("- Barbell has higher convexity than bullet for similar duration")
print("- Ladder provides reinvestment diversification")
print("- Higher convexity = better performance for large yield changes")

In [ ]:
# Visualize Portfolio Strategies Performance
yield_shocks = np.linspace(-0.02, 0.02, 50)

def portfolio_value_change(portfolio, base_ytm, yield_shock):
    """Calculate portfolio value change for a parallel yield shift."""
    base_value = portfolio.portfolio_value()
    
    # Create new portfolio with shocked yields
    new_value = sum(
        bond.price(base_ytm + yield_shock) * qty
        for bond, qty in zip(portfolio.bonds, portfolio.quantities)
    )
    
    return (new_value - base_value) / base_value * 100

fig = go.Figure()

colors = {'Bullet (10yr)': '#3498db', 'Barbell (2yr/30yr)': '#e74c3c', 'Ladder (2-30yr)': '#27ae60'}

for name, port in portfolios:
    changes = [portfolio_value_change(port, common_ytm, shock) for shock in yield_shocks]
    fig.add_trace(go.Scatter(
        x=yield_shocks * 100, y=changes,
        mode='lines', name=name,
        line=dict(color=colors[name], width=2)
    ))

fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.add_vline(x=0, line_dash="dash", line_color="gray")

fig.update_layout(
    title='Portfolio Value Change Under Parallel Yield Shifts',
    xaxis_title='Yield Change (basis points × 100)',
    yaxis_title='Portfolio Value Change (%)',
    template='plotly_white',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    height=500
)

fig

---
## 7. Credit Risk in Fixed Income

### Credit Ratings

| Moody's | S&P | Fitch | Grade | Risk Level |
|---------|-----|-------|-------|------------|
| Aaa | AAA | AAA | Investment | Highest quality |
| Aa1-Aa3 | AA+/AA/AA- | AA+/AA/AA- | Investment | High quality |
| A1-A3 | A+/A/A- | A+/A/A- | Investment | Upper medium |
| Baa1-Baa3 | BBB+/BBB/BBB- | BBB+/BBB/BBB- | Investment | Medium grade |
| Ba1-Ba3 | BB+/BB/BB- | BB+/BB/BB- | Speculative | Speculative |
| B1-B3 | B+/B/B- | B+/B/B- | Speculative | Highly speculative |
| Caa-C | CCC-D | CCC-D | Speculative | Near/in default |

### Credit Spreads

The credit spread is the yield premium over a risk-free benchmark:

$$\text{Credit Spread} = Y_{\text{corporate}} - Y_{\text{treasury}}$$

### Spread Measures

#### G-Spread (Government Spread)
Spread over the interpolated government bond yield.

#### I-Spread (Interpolated Spread)
Spread over the swap curve.

#### Z-Spread (Zero-volatility Spread)
Constant spread added to the spot curve to reprice the bond:

$$P = \sum_{t=1}^{n} \frac{CF_t}{(1 + z_t + Z)^t}$$

#### OAS (Option-Adjusted Spread)
Z-spread adjusted for embedded options.

### Default Probability

The risk-neutral default probability can be estimated from credit spreads:

$$\text{Default Probability} \approx \frac{\text{Credit Spread}}{1 - \text{Recovery Rate}}$$

In [ ]:
# Credit Spread Analysis
print("=" * 60)
print("CREDIT RISK ANALYSIS")
print("=" * 60)

# Typical credit spreads by rating (in basis points)
credit_spreads = {
    'AAA': 20,
    'AA': 40,
    'A': 70,
    'BBB': 120,
    'BB': 250,
    'B': 450,
    'CCC': 800
}

# Historical default rates by rating (annual, in %)
default_rates = {
    'AAA': 0.00,
    'AA': 0.02,
    'A': 0.06,
    'BBB': 0.18,
    'BB': 0.81,
    'B': 4.42,
    'CCC': 26.78
}

# Assumed recovery rate
recovery_rate = 0.40

print("\n📊 Credit Spreads and Default Risk:")
print("-" * 70)
print(f"{'Rating':>8} {'Spread (bps)':>15} {'Historical DR':>15} {'Implied DR':>15}")
print("-" * 70)

for rating in credit_spreads:
    spread = credit_spreads[rating]
    hist_dr = default_rates[rating]
    implied_dr = (spread / 10000) / (1 - recovery_rate) * 100  # Convert to percentage
    print(f"{rating:>8} {spread:>15} {hist_dr:>14.2f}% {implied_dr:>14.2f}%")

print(f"\n💡 Note: Implied default rates assume {recovery_rate*100:.0f}% recovery rate")

In [ ]:
# Visualize Credit Spreads
ratings = list(credit_spreads.keys())
spreads = list(credit_spreads.values())
defaults = list(default_rates.values())

# Create color scale based on risk
colors = ['#27ae60', '#2ecc71', '#f1c40f', '#f39c12', '#e67e22', '#e74c3c', '#c0392b']

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Credit Spreads by Rating', 'Historical Default Rates'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

fig.add_trace(
    go.Bar(
        x=ratings, y=spreads,
        marker_color=colors,
        text=spreads,
        textposition='outside',
        name='Spread (bps)'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=ratings, y=defaults,
        marker_color=colors,
        text=[f'{d:.2f}%' for d in defaults],
        textposition='outside',
        name='Default Rate (%)'
    ),
    row=1, col=2
)

fig.update_yaxes(title_text='Spread (bps)', row=1, col=1)
fig.update_yaxes(title_text='Default Rate (%)', type='log', row=1, col=2)

fig.update_layout(
    title='Credit Risk Analysis by Rating',
    template='plotly_white',
    height=450,
    showlegend=False
)

fig

In [ ]:
def calculate_z_spread(bond, market_price, spot_curve, tolerance=1e-6):
    """
    Calculate Z-Spread for a bond.
    
    Parameters:
    -----------
    bond : Bond
        Bond object
    market_price : float
        Current market price of the bond
    spot_curve : YieldCurve
        Spot rate curve
    tolerance : float
        Convergence tolerance
    
    Returns:
    --------
    float : Z-Spread in decimal form
    """
    def price_with_spread(z_spread):
        price = 0
        for t in range(1, bond.periods + 1):
            time_years = t / bond.frequency
            spot = spot_curve.interpolate_rate(time_years)
            
            if t < bond.periods:
                cf = bond.coupon_payment
            else:
                cf = bond.coupon_payment + bond.face_value
            
            price += cf / (1 + (spot + z_spread) / bond.frequency)**t
        return price
    
    def objective(z):
        return price_with_spread(z) - market_price
    
    return brentq(objective, -0.5, 1.0)


# Example: Z-Spread calculation
print("\n📊 Z-Spread Calculation Example:")
print("-" * 50)

# Corporate bond trading at a discount
corp_bond = Bond(1000, 0.05, 5, 2)  # 5% coupon, 5-year
corp_market_price = 980  # Trading below par

# Calculate Z-spread over the normal curve
z_spread = calculate_z_spread(corp_bond, corp_market_price, normal_curve)

# For comparison, calculate G-spread
corp_ytm = corp_bond.ytm(corp_market_price)
treasury_yield = normal_curve.interpolate_rate(5)  # 5-year Treasury yield
g_spread = corp_ytm - treasury_yield

print(f"Corporate Bond: 5% coupon, 5-year maturity")
print(f"Market Price: ${corp_market_price}")
print(f"Corporate YTM: {corp_ytm*100:.4f}%")
print(f"5-Year Treasury Yield: {treasury_yield*100:.4f}%")
print(f"G-Spread: {g_spread*10000:.1f} basis points")
print(f"Z-Spread: {z_spread*10000:.1f} basis points")

---
## 8. Interest Rate Risk Hedging

### Duration Matching

To hedge interest rate risk, match the dollar duration of assets and liabilities:

$$D_A \cdot V_A = D_L \cdot V_L$$

### Hedge Ratio

The hedge ratio determines the number of hedging instruments needed:

$$HR = \frac{DV01_{\text{portfolio}}}{DV01_{\text{hedge instrument}}}$$

### Key Rate Duration (KRD)

Measures sensitivity to changes in specific points on the yield curve:

$$KRD_t = -\frac{1}{P} \cdot \frac{\partial P}{\partial r_t}$$

### Hedging Instruments

| Instrument | Use Case | Advantages |
|------------|----------|------------|
| **Treasury Futures** | Duration hedge | Liquid, standardized |
| **Interest Rate Swaps** | Asset-liability matching | Customizable |
| **Eurodollar Futures** | Short-term rate hedge | Liquid |
| **Treasury Options** | Convexity hedge | Asymmetric payoff |
| **Swaptions** | Option on swaps | Flexible |

### Futures Hedge Example

Number of futures contracts:

$$N = \frac{D_{\text{portfolio}} \cdot V_{\text{portfolio}}}{D_{\text{futures}} \cdot V_{\text{futures}}} \cdot \beta$$

Where $\beta$ is the yield beta (sensitivity of portfolio yield to CTD yield).

In [ ]:
def calculate_hedge_ratio(portfolio, hedge_instrument_bond, hedge_ytm):
    """
    Calculate the number of hedge instruments needed to neutralize duration.
    
    Parameters:
    -----------
    portfolio : BondPortfolio
        The bond portfolio to hedge
    hedge_instrument_bond : Bond
        The bond used as hedge instrument (e.g., Treasury bond)
    hedge_ytm : float
        YTM of the hedge instrument
    
    Returns:
    --------
    dict : Hedge ratio and related metrics
    """
    # Portfolio DV01
    port_dv01 = portfolio.portfolio_dv01()
    
    # Hedge instrument DV01
    hedge_dur = calculate_duration(hedge_instrument_bond, hedge_ytm)
    hedge_dv01 = hedge_dur['dv01']
    hedge_price = hedge_dur['price']
    
    # Hedge ratio (number of bonds needed)
    hedge_ratio = port_dv01 / hedge_dv01
    
    # Value of hedge
    hedge_value = hedge_ratio * hedge_price
    
    return {
        'portfolio_dv01': port_dv01,
        'hedge_dv01': hedge_dv01,
        'hedge_ratio': hedge_ratio,
        'hedge_price': hedge_price,
        'hedge_value': hedge_value
    }


def key_rate_duration(bond, base_ytm, key_rate_maturity, shift=0.0001):
    """
    Calculate key rate duration for a specific maturity point.
    
    This is a simplified version - in practice, you would shift the
    spot curve at a specific key rate and interpolate effects.
    
    Parameters:
    -----------
    bond : Bond
        Bond object
    base_ytm : float
        Base yield to maturity
    key_rate_maturity : float
        Maturity point to shift
    shift : float
        Size of rate shift
    
    Returns:
    --------
    float : Key rate duration
    """
    # Sensitivity decays with distance from key rate
    distance = abs(bond.years_to_maturity - key_rate_maturity)
    decay = max(0, 1 - distance / bond.years_to_maturity) if bond.years_to_maturity > 0 else 0
    
    # Effective duration contribution
    eff_dur = calculate_effective_duration(bond, base_ytm)
    
    return eff_dur * decay

In [ ]:
# Example: Duration Hedging
print("=" * 70)
print("INTEREST RATE HEDGING ANALYSIS")
print("=" * 70)

# Create a portfolio to hedge
portfolio_to_hedge = BondPortfolio()
portfolio_to_hedge.add_bond(Bond(1000, 0.06, 15, 2), 100, 0.055)  # Corporate bonds
portfolio_to_hedge.add_bond(Bond(1000, 0.055, 10, 2), 50, 0.05)   # More corporate bonds

# Hedge instrument: 10-year Treasury
treasury_hedge = Bond(1000, 0.04, 10, 2)
treasury_ytm = 0.045

# Calculate hedge ratio
hedge_result = calculate_hedge_ratio(portfolio_to_hedge, treasury_hedge, treasury_ytm)

print("\n📊 Portfolio to Hedge:")
print("-" * 50)
print(portfolio_to_hedge.summary().to_string(index=False))

print(f"\n📈 Hedge Calculation:")
print("-" * 50)
print(f"Portfolio Value:     ${portfolio_to_hedge.portfolio_value():,.2f}")
print(f"Portfolio Duration:  {portfolio_to_hedge.portfolio_duration():.4f} years")
print(f"Portfolio DV01:      ${hedge_result['portfolio_dv01']:,.2f}")
print(f"\nHedge Instrument (10yr Treasury):")
print(f"  Price:            ${hedge_result['hedge_price']:.2f}")
print(f"  DV01:             ${hedge_result['hedge_dv01']:.4f}")
print(f"\n🎯 Hedge Ratio:      {hedge_result['hedge_ratio']:.2f} bonds")
print(f"   (SHORT {abs(hedge_result['hedge_ratio']):.0f} Treasury bonds to hedge)")
print(f"   Hedge Value:      ${hedge_result['hedge_value']:,.2f}")

In [ ]:
# Visualize Hedge Effectiveness
yield_changes = np.linspace(-0.015, 0.015, 50)

# Unhedged portfolio P&L
unhedged_pnl = []
hedged_pnl = []

base_portfolio_value = portfolio_to_hedge.portfolio_value()
base_hedge_value = hedge_result['hedge_ratio'] * treasury_hedge.price(treasury_ytm)

for dy in yield_changes:
    # Portfolio value change (assuming parallel shift)
    new_port_value = sum(
        bond.price(ytm + dy) * qty
        for bond, qty, ytm in zip(
            portfolio_to_hedge.bonds,
            portfolio_to_hedge.quantities,
            portfolio_to_hedge.ytms
        )
    )
    port_pnl = new_port_value - base_portfolio_value
    unhedged_pnl.append(port_pnl)
    
    # Hedge P&L (short position gains when prices fall)
    new_hedge_value = hedge_result['hedge_ratio'] * treasury_hedge.price(treasury_ytm + dy)
    hedge_pnl = -(new_hedge_value - base_hedge_value)  # Short position
    
    hedged_pnl.append(port_pnl + hedge_pnl)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=yield_changes * 10000, y=np.array(unhedged_pnl) / 1000,
    mode='lines', name='Unhedged Portfolio',
    line=dict(color='#e74c3c', width=2)
))

fig.add_trace(go.Scatter(
    x=yield_changes * 10000, y=np.array(hedged_pnl) / 1000,
    mode='lines', name='Duration-Hedged Portfolio',
    line=dict(color='#27ae60', width=2)
))

fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.add_vline(x=0, line_dash="dash", line_color="gray")

fig.update_layout(
    title='Hedge Effectiveness: Unhedged vs Duration-Hedged Portfolio',
    xaxis_title='Yield Change (basis points)',
    yaxis_title='P&L ($ thousands)',
    template='plotly_white',
    legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
    height=500
)

fig

---
## 9. Comprehensive Examples

Let's bring together all concepts with comprehensive, real-world examples.

In [ ]:
# Comprehensive Example: Bond Analysis Dashboard
print("=" * 80)
print("COMPREHENSIVE BOND ANALYSIS DASHBOARD")
print("=" * 80)

# Create a sample bond for comprehensive analysis
sample_bond = Bond(
    face_value=1000,
    coupon_rate=0.055,  # 5.5% coupon
    years_to_maturity=15,
    frequency=2
)

market_ytm = 0.05  # Current market yield is 5%

# Calculate all metrics
price = sample_bond.price(market_ytm)
current_yld = sample_bond.current_yield(price)
duration_metrics = calculate_duration(sample_bond, market_ytm)
effective_dur = calculate_effective_duration(sample_bond, market_ytm)
convexity = calculate_convexity(sample_bond, market_ytm)

print("\n📊 BOND CHARACTERISTICS")
print("-" * 50)
print(f"Face Value:          ${sample_bond.face_value:,.2f}")
print(f"Coupon Rate:         {sample_bond.coupon_rate*100:.2f}%")
print(f"Coupon Payment:      ${sample_bond.coupon_payment:.2f} (semi-annual)")
print(f"Years to Maturity:   {sample_bond.years_to_maturity} years")
print(f"Payment Frequency:   Semi-annual")

print("\n📈 PRICING & YIELD METRICS")
print("-" * 50)
print(f"Market YTM:          {market_ytm*100:.2f}%")
print(f"Bond Price:          ${price:.4f}")
print(f"Price as % of Par:   {price/10:.2f}%")
print(f"Current Yield:       {current_yld*100:.4f}%")
print(f"Premium/Discount:    {'Premium' if price > 1000 else 'Discount' if price < 1000 else 'Par'}")

print("\n⏱️ DURATION METRICS")
print("-" * 50)
print(f"Macaulay Duration:   {duration_metrics['macaulay_duration']:.4f} years")
print(f"Modified Duration:   {duration_metrics['modified_duration']:.4f} years")
print(f"Effective Duration:  {effective_dur:.4f} years")
print(f"DV01 (per bond):     ${duration_metrics['dv01']:.4f}")

print("\n📐 CONVEXITY")
print("-" * 50)
print(f"Convexity:           {convexity:.4f} years²")
print(f"Dollar Convexity:    ${convexity * price:.2f}")

In [ ]:
# Scenario Analysis
print("\n📊 SCENARIO ANALYSIS")
print("-" * 80)

scenarios = [
    ('Rates fall 100bp', -0.01),
    ('Rates fall 50bp', -0.005),
    ('Rates fall 25bp', -0.0025),
    ('No change', 0),
    ('Rates rise 25bp', 0.0025),
    ('Rates rise 50bp', 0.005),
    ('Rates rise 100bp', 0.01),
]

print(f"{'Scenario':>22} {'New Yield':>12} {'New Price':>12} {'Price Δ':>12} {'Price Δ %':>12}")
print("-" * 80)

scenario_data = []
for scenario_name, dy in scenarios:
    new_ytm = market_ytm + dy
    new_price = sample_bond.price(new_ytm)
    price_change = new_price - price
    pct_change = (new_price - price) / price * 100
    
    print(f"{scenario_name:>22} {new_ytm*100:>11.2f}% ${new_price:>10.2f} ${price_change:>+10.2f} {pct_change:>+11.2f}%")
    
    scenario_data.append({
        'Scenario': scenario_name,
        'Yield Change': dy * 10000,
        'New Price': new_price,
        'Price Change %': pct_change
    })

In [ ]:
# Create comprehensive visualization dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Price-Yield Curve',
        'Duration vs Convexity Effect',
        'Cash Flow Profile',
        'Scenario Analysis'
    ),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"type": "bar"}]]
)

# Plot 1: Price-Yield Curve
yields_viz = np.linspace(0.02, 0.10, 100)
prices_viz = [sample_bond.price(y) for y in yields_viz]

fig.add_trace(
    go.Scatter(x=yields_viz*100, y=prices_viz, mode='lines',
               name='Price-Yield', line=dict(color='#3498db', width=2)),
    row=1, col=1
)

# Add current point
fig.add_trace(
    go.Scatter(x=[market_ytm*100], y=[price], mode='markers',
               name='Current', marker=dict(size=12, color='#e74c3c')),
    row=1, col=1
)

# Plot 2: Duration vs Convexity approximation
dy_range = np.linspace(-0.02, 0.02, 100)
mod_dur = duration_metrics['modified_duration']

actual_changes = [(sample_bond.price(market_ytm + dy) - price) / price * 100 for dy in dy_range]
duration_approx = [-mod_dur * dy * 100 for dy in dy_range]
convex_approx = [(-mod_dur * dy + 0.5 * convexity * dy**2) * 100 for dy in dy_range]

fig.add_trace(
    go.Scatter(x=dy_range*100, y=actual_changes, mode='lines',
               name='Actual', line=dict(color='#2c3e50', width=2)),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=dy_range*100, y=duration_approx, mode='lines',
               name='Duration Only', line=dict(color='#e74c3c', width=2, dash='dash')),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=dy_range*100, y=convex_approx, mode='lines',
               name='Duration+Convexity', line=dict(color='#27ae60', width=2, dash='dot')),
    row=1, col=2
)

# Plot 3: Cash Flow Profile
cf_times = np.arange(0.5, sample_bond.years_to_maturity + 0.5, 0.5)
cash_flows = [sample_bond.coupon_payment] * (len(cf_times) - 1) + \
             [sample_bond.coupon_payment + sample_bond.face_value]

fig.add_trace(
    go.Bar(x=cf_times, y=cash_flows, name='Cash Flows',
           marker_color='#9b59b6'),
    row=2, col=1
)

# Plot 4: Scenario Analysis
scenario_df = pd.DataFrame(scenario_data)
colors_scenario = ['#27ae60' if x > 0 else '#e74c3c' for x in scenario_df['Price Change %']]

fig.add_trace(
    go.Bar(x=scenario_df['Scenario'], y=scenario_df['Price Change %'],
           name='Price Change', marker_color=colors_scenario),
    row=2, col=2
)

# Update layout
fig.update_xaxes(title_text='Yield (%)', row=1, col=1)
fig.update_xaxes(title_text='Yield Change (%)', row=1, col=2)
fig.update_xaxes(title_text='Time (Years)', row=2, col=1)
fig.update_xaxes(title_text='', tickangle=45, row=2, col=2)

fig.update_yaxes(title_text='Price ($)', row=1, col=1)
fig.update_yaxes(title_text='Price Change (%)', row=1, col=2)
fig.update_yaxes(title_text='Cash Flow ($)', row=2, col=1)
fig.update_yaxes(title_text='Price Change (%)', row=2, col=2)

fig.update_layout(
    title=f'Bond Analysis Dashboard: {sample_bond.coupon_rate*100}% {sample_bond.years_to_maturity}-Year Bond',
    template='plotly_white',
    height=800,
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5)
)

fig

In [ ]:
# Interactive Yield Curve Builder
print("\n" + "=" * 70)
print("YIELD CURVE ANALYSIS")
print("=" * 70)

# Create realistic yield curves for different economic scenarios
maturities_full = np.array([0.25, 0.5, 1, 2, 3, 5, 7, 10, 20, 30])

# Current environment
current_rates = np.array([0.045, 0.047, 0.048, 0.046, 0.044, 0.042, 0.041, 0.040, 0.042, 0.043])

# Steepening scenario
steep_rates = np.array([0.030, 0.032, 0.035, 0.040, 0.045, 0.050, 0.053, 0.055, 0.058, 0.060])

# Flattening scenario  
flat_rates = np.array([0.048, 0.048, 0.047, 0.046, 0.045, 0.044, 0.043, 0.042, 0.042, 0.042])

curves = {
    'Current (Inverted)': YieldCurve(maturities_full, current_rates),
    'Steepening': YieldCurve(maturities_full, steep_rates),
    'Flattening': YieldCurve(maturities_full, flat_rates)
}

# Calculate curve metrics
print("\n📊 Yield Curve Metrics:")
print("-" * 70)
print(f"{'Scenario':>20} {'2Y Rate':>10} {'10Y Rate':>10} {'2s10s Spread':>15} {'Slope':>12}")
print("-" * 70)

for name, curve in curves.items():
    r2 = curve.interpolate_rate(2) * 100
    r10 = curve.interpolate_rate(10) * 100
    spread = (r10 - r2) * 100  # In basis points
    slope = "Inverted" if spread < 0 else "Normal" if spread > 50 else "Flat"
    print(f"{name:>20} {r2:>9.2f}% {r10:>9.2f}% {spread:>+14.0f}bp {slope:>12}")

In [ ]:
# Visualize yield curves
fig = go.Figure()

mat_interp = np.linspace(0.25, 30, 100)
colors_curves = {'Current (Inverted)': '#e74c3c', 'Steepening': '#27ae60', 'Flattening': '#3498db'}

for name, curve in curves.items():
    rates_interp = [curve.interpolate_rate(t) * 100 for t in mat_interp]
    fig.add_trace(go.Scatter(
        x=mat_interp, y=rates_interp,
        mode='lines', name=name,
        line=dict(color=colors_curves[name], width=2)
    ))

# Add markers for key maturities
key_mats = [2, 5, 10, 30]
for name, curve in curves.items():
    rates_key = [curve.interpolate_rate(t) * 100 for t in key_mats]
    fig.add_trace(go.Scatter(
        x=key_mats, y=rates_key,
        mode='markers', name=f'{name} Key Rates',
        marker=dict(size=8, color=colors_curves[name]),
        showlegend=False
    ))

fig.update_layout(
    title='Yield Curve Scenarios',
    xaxis_title='Maturity (Years)',
    yaxis_title='Yield (%)',
    template='plotly_white',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    height=500
)

fig

---
## Key Takeaways

### 1. Bond Pricing
- Bond prices are the present value of future cash flows
- Prices move inversely to yields
- Premium bonds have coupon > YTM; discount bonds have coupon < YTM

### 2. Duration
- **Macaulay Duration**: Weighted average time to receive cash flows
- **Modified Duration**: Price sensitivity to yield changes
- **Effective Duration**: Best for bonds with embedded options
- Higher duration = higher interest rate risk

### 3. Convexity
- Measures curvature in price-yield relationship
- Positive convexity benefits bondholders
- Always include convexity adjustment for large yield changes

### 4. Term Structure
- Spot rates are yields on zero-coupon bonds
- Forward rates are implied future rates
- Inverted yield curves often precede recessions

### 5. Credit Risk
- Credit spreads compensate for default risk
- Higher ratings = lower spreads = lower default probability
- Z-spread provides better credit risk measure than G-spread

### 6. Risk Management
- Duration matching hedges against parallel yield curve shifts
- Key rate durations hedge against non-parallel shifts
- Barbell portfolios have higher convexity than bullet portfolios

---

## Further Reading

- Fabozzi, F. J. (2016). *Bond Markets, Analysis, and Strategies* (9th ed.)
- Tuckman, B., & Serrat, A. (2012). *Fixed Income Securities* (3rd ed.)
- Hull, J. C. (2018). *Options, Futures, and Other Derivatives* (10th ed.)
- CFA Institute. *Fixed Income Analysis*